In [3]:
#!pip install dash dash-extensions


In [14]:
from dash import Dash, dcc, html, Input, Output, callback, clientside_callback, State
import plotly.graph_objects as go
import forecast
import pandas as pd

start_date = '2024-01-02 00:00'
candles_df = forecast.get_df('BTCUSD', start_date, '1T')

def create_figure(df, relayout_store=None):
    df = candles_df[0:100].copy()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=df['high'] * 1.001, mode='lines', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df.index, y=df['low'] * 0.999, mode='lines', line=dict(color='steelblue')))
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['close'],
                                 name='Candlesticks'))
    fig.update_layout(dragmode='pan', xaxis_rangeslider_visible=False)
    
    if relayout_store:
        if 'xaxis.range[0]' in relayout_store and 'xaxis.range[1]' in relayout_store:
            fig.update_layout(xaxis=dict(range=[relayout_store['xaxis.range[0]'], relayout_store['xaxis.range[1]']]))
        if 'yaxis.range[0]' in relayout_store and 'yaxis.range[1]' in relayout_store:
            fig.update_layout(yaxis=dict(range=[relayout_store['yaxis.range[0]'], relayout_store['yaxis.range[1]']]))
        if 'dragmode' in relayout_store:
            fig.update_layout(dragmode=relayout_store['dragmode'])

    return fig

app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='basic_interactions'),
    dcc.Store(id='relayout_store'),
    html.Button('Clear', id='clr'),
    html.Div(id='blank-output')
])

@callback(
    Output('relayout_store', 'data'),
    Input('basic_interactions', 'relayoutData'),
    State('relayout_store', 'data'))
def on_relayoutData(relayoutData, relayout_store):
    if relayoutData is None:
        return relayout_store

    if relayout_store is None:
        relayout_store = {}

    if 'dragmode' in relayoutData:
        relayout_store['dragmode'] = relayoutData['dragmode']
    if 'xaxis.range[0]' in relayoutData:
        relayout_store['xaxis.range[0]'] = relayoutData['xaxis.range[0]']
    if 'xaxis.range[1]' in relayoutData:
        relayout_store['xaxis.range[1]'] = relayoutData['xaxis.range[1]']
    if 'yaxis.range[0]' in relayoutData:
        relayout_store['yaxis.range[0]'] = relayoutData['yaxis.range[0]']
    if 'yaxis.range[1]' in relayoutData:
        relayout_store['yaxis.range[1]'] = relayoutData['yaxis.range[1]']
    if 'xaxis.autorange' in relayoutData or 'autosize' in relayoutData:
        relayout_store.pop('xaxis.range[0]', None)
        relayout_store.pop('xaxis.range[1]', None)
    if 'yaxis.autorange' in relayoutData or 'autosize' in relayoutData:
        relayout_store.pop('yaxis.range[0]', None)
        relayout_store.pop('yaxis.range[1]', None)

    return relayout_store

@callback(
    Output('basic_interactions', 'figure'),
    Input('relayout_store', 'data'))
def update_graph(relayout_store):
    return create_figure(candles_df, relayout_store)

clientside_callback(
    """
function(fig) {
    const graphDiv = document.getElementById('basic_interactions').getElementsByClassName('js-plotly-plot')[0];
    if (graphDiv) {
        graphDiv.onwheel = function(event) {
            //debugger;
            event.preventDefault();
            
            const zoomLevel = 0.95; // Zoom out 5%
            const { xaxis, yaxis } = graphDiv.layout;

            console.log('layout:');
            console.log(graphDiv.layout);
            console.log('xaxis.range: ' + xaxis.range);

            // Convert date string to UTC
            const convertToUTC = dateStr => dateStr.length === 10 ? `${dateStr}T00:00:00Z` : dateStr.split(' ').join('T') + 'Z';

            // Parse date strings to Date objects
            const xrange = xaxis.range.map(x => new Date(Date.parse(convertToUTC(x))));
            console.log('xrange: ' + xrange);
            const yrange = yaxis.range;
            console.log(yrange);

            // Calculate the zoom delta
            const dx = (xrange[1] - xrange[0]) * (1 - zoomLevel) / 2;
            const dy = (yrange[1] - yrange[0]) * (1 - zoomLevel) / 2;
            console.log('dx: ' + dx);
            console.log('dy: ' + dy);

            // Determine zoom direction
            const zoom = event.deltaY < 0 ? 1 : -1;

            let newX0date, newX1date, newX0, newX1;
            if (event.ctrlKey) {
                // Zoom around cursor position
                const cursorX = event.offsetX / graphDiv.clientWidth;
                const zoomDelta = (xrange[1] - xrange[0]) * (1 - zoomLevel);
                newX0date = new Date(xrange[0].getTime() + zoom * cursorX * zoomDelta);
                newX0 = newX0date.toISOString().split('T').join(' ').replace('Z', '');
                newX1date = new Date(xrange[1].getTime() - zoom * (1 - cursorX) * zoomDelta);
                newX1 = newX1date.toISOString().split('T').join(' ').replace('Z', '');
            } else {
                // Zoom with right edge fixed
                newX0date = new Date(xrange[0].getTime() + zoom * dx);
                newX0 = newX0date.toISOString().split('T').join(' ').replace('Z', '');
                newX1date = new Date(xrange[1].getTime());
                newX1 = xaxis.range[1];
            }

           // Compute new y range based on new x range
            const newYRanges = graphDiv.data.map(trace => {
                const xValues = trace.x.map(x => new Date(Date.parse(convertToUTC(x))));
                let yMin, yMax;
                if (trace.y) {
                    const yValues = trace.y;
                    const withinRange = yValues.filter((y, i) => xValues[i] >= newX0date && xValues[i] <= newX1date);
                    yMax = Math.max(...withinRange);
                    yMin = Math.min(...withinRange);
                }
                else {
                    let yValues = trace.high;
                    let withinRange = yValues.filter((y, i) => xValues[i] >= newX0date && xValues[i] <= newX1date);
                    yMax = Math.max(...withinRange);
                    yValues = trace.low;
                    withinRange = yValues.filter((y, i) => xValues[i] >= newX0date && xValues[i] <= newX1date);
                    yMin = Math.min(...withinRange);
                }
                const yPadding = (yMax - yMin) * 0.05; // 5%
                yMin = yMin - yPadding;
                yMax = yMax + yPadding;
                return [yMin, yMax];
            });

            const newY0 = Math.min(...newYRanges.map(range => range[0]));
            const newY1 = Math.max(...newYRanges.map(range => range[1]));

            console.log('new range y: ' + newY0 + ' ... ' + newY1);

            // Apply new ranges
            Plotly.relayout(graphDiv, {
                'xaxis.range[0]': newX0,
                'xaxis.range[1]': newX1,
                'yaxis.range[0]': newY0,
                'yaxis.range[1]': newY1,
            });

        };
    }
    return window.dash_clientside.no_update;
}

    """,
    Output('blank-output', 'children'),
    Input('basic_interactions', 'figure')
)

if __name__ == "__main__":
    app.run_server(debug=True)
